<a href="https://colab.research.google.com/github/annabelcoates/redev_machine_learning_samples/blob/master/FootballPredicator/fp_train_odds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries


In [3]:
import sys
import os
import urllib.request
import datetime;
import pandas as pd
import numpy as np
import math

from keras import optimizers
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils, to_categorical
from keras.utils import plot_model


Using TensorFlow backend.


# Load preprocessed data 

For data without categorical names use preprocessed_data.csv as the filename or preprocessed_data_Cat.csv for the (220 columns of) categoric names

In [0]:

url='https://raw.githubusercontent.com/RedevLtd/redev_machine_learning_samples/master/FootballPredicator/preprocessed_data/preprocessed_data.csv'
df=pd.read_csv(url)
df_array=np.asarray(df)


In [5]:
count=0
for column in df.columns:
  print(str(count) +') '+ str(column))
  count+=1
  

0) Div
1) MatchNo
2) SeasonYear
3) SeasonMatchNo
4) MatchDate
5) HomeTeam
6) AwayTeam
7) OddsHomeWin
8) OddsDraw
9) OddsAwayWin
10) OddsWinDiff
11) Attendance
12) L_FTHomeGoals
13) L_FTAwayGoals
14) L_FTResult
15) L_FTHomeWin
16) L_FTDraw
17) L_FTAwayWin
18) L_HTHomeGoals
19) L_HTAwayGoals
20) L_HTResult
21) L_HTHomeWin
22) L_HTDraw
23) L_HTAwayWin


In [6]:
df.head(3)

,Div,MatchNo,SeasonYear,SeasonMatchNo,MatchDate,HomeTeam,AwayTeam,OddsHomeWin,OddsDraw,OddsAwayWin,OddsWinDiff,Attendance,L_FTHomeGoals,L_FTAwayGoals,L_FTResult,L_FTHomeWin,L_FTDraw,L_FTAwayWin,L_HTHomeGoals,L_HTAwayGoals,L_HTResult,L_HTHomeWin,L_HTDraw,L_HTAwayWin
0,E0,20000001,2000,1,2000-08-19,Charlton,Man City,2.10,3.2,3.10,1.00,20043.0,4.0,0.0,H,1,0,0,2.0,0.0,H,1,0,0
1,E0,20000002,2000,2,2000-08-19,Chelsea,West Ham,1.44,3.6,6.50,5.06,34914.0,4.0,2.0,H,1,0,0,1.0,0.0,H,1,0,0
2,E0,20000003,2000,3,2000-08-19,Coventry,Middlesbrough,2.30,3.2,2.62,0.32,20624.0,1.0,3.0,A,0,0,1,1.0,1.0,D,0,1,0


In [0]:
input_array=df_array[:,[7,8,9]] # The HDA odds

results=df_array[:,[15,16,17]] 

In [0]:
# Get the number of inputs and the number of datapoints
number_dp=input_array.shape[0]
n_inputs= input_array.shape[1] # For robustness, in this case the number of inputs is 3 for the 3 odds

In [0]:
input_array=input_array.reshape(number_dp,1,1,n_inputs)
results=results.reshape(number_dp,1,1,3)

# Seperate into training and testing data

Usually approx 80% of the data is used as training data and 20% of the data is used as test data

In [0]:
p=0.8
last_train= math.ceil(p*number_dp)
# Training data
input_train=input_array[0:last_train,:]
results_train=results[0:last_train,:]
# Test data
input_test=input_array[(last_train+1):number_dp,:]
results_test=results[(last_train+1):number_dp,:]

In [11]:
print(input_test.shape)
print(results_test.shape)

(7558, 1, 1, 3)
(7558, 1, 1, 3)


Print input shape

# Define Model Architecture

In [0]:
model=Sequential()
#model.add(Dropout(0.1)) 
model.add(Dense(3,input_shape=(1,1,n_inputs), activation='relu')) # Adds a densely connected NN layer
model.add(Dense(3,activation='relu')) 
model.add(Dense(3,activation='softmax'))

# Compile Model

Categorial crossentropy loss is a loss algorithm for when there are multiple categories to sort each data point into, in this case  home win, draw or away win. The optimizer is a feature that makes the model work better, adam is the default.

In [0]:
#opt = SGD(lr=0.000001,momentum=0.004)
model.compile(loss='categorical_crossentropy',optimizer='adagrad',metrics=['accuracy']) 
# Want to know the accuracy of the model so specify it as a metric
plot_model(model,to_file='model.png')



# Fit Model from Training Data


* The number of epochs are the number of phases of training. 
* The shuffle parameter determines whether the training data is shuffled between epochs. 
* Batch size is the number of data points between updates to the weights.
* **Lower batch size** and a **greater number of epochs** can increase accuracy



In [20]:


model.fit(input_train,results_train,batch_size=500,epochs=50,verbose=1,shuffle=1)

Epoch 1/50
30237/30237 [==============================] - 1s 30us/step - loss: 1.0540 - acc: 0.4683
Epoch 2/50
30237/30237 [==============================] - 0s 16us/step - loss: 1.0371 - acc: 0.4725
Epoch 3/50
30237/30237 [==============================] - 0s 14us/step - loss: 1.0365 - acc: 0.4722
Epoch 4/50
30237/30237 [==============================] - 0s 15us/step - loss: 1.0363 - acc: 0.4721
Epoch 5/50
30237/30237 [==============================] - 0s 14us/step - loss: 1.0361 - acc: 0.4720
Epoch 6/50
30237/30237 [==============================] - 0s 14us/step - loss: 1.0360 - acc: 0.4718
Epoch 7/50
30237/30237 [==============================] - 0s 14us/step - loss: 1.0359 - acc: 0.4729
Epoch 8/50
30237/30237 [==============================] - 0s 13us/step - loss: 1.0358 - acc: 0.4732
Epoch 9/50
30237/30237 [==============================] - 0s 15us/step - loss: 1.0358 - acc: 0.4716
Epoch 10/50
30237/30237 [==============================] - 0s 14us/step - loss: 1.0358 - acc: 0.4728

# Evaluate Model

In [21]:
[loss_value,accuracy]=model.evaluate(input_test,results_test,verbose=1)
print("The loss value for the model is "+ str(loss_value))
print("The accuracy value for the model is "+"{:.3f}".format(accuracy))


7558/7558 [==============================] - 1s 79us/step
The loss value for the model is 1.0247916758896403
The accuracy value for the model is 0.486


# Make Prediction

# Predict Class

In [22]:
h=float(input('Enter odds of home team win: '))
d=float(input('Enter odds of draw: '))
a=float(input('Enter odds of away team win: '))
odds_input=np.asarray([h,d,a]).reshape(1,1,1,3)
predicted_outcome=model.predict_classes(odds_input)
if predicted_outcome==0:
  result = 'the home team will win'
elif predicted_outcome==1:
  result= 'the match will be a draw'
elif predicted_outcome==2:
  result='the away team will win'
print('The model predicts that '+result)

Enter odds of home team win: 1.8
Enter odds of draw: 4
Enter odds of away team win: 5
The model predicts that the home team will win
